# 轴线沿着x的圆锥

* 先计算最大或者最小位置，然后你和出来一条线
* 计算距离这条线n纳米以内的
* 然后展开数据
* 数据的维度怎么计算。

In [1]:
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [2]:
from analyze import curl
import numpy as np

In [12]:
f=r'C:\Users\omf\Desktop\mpy1.0\test\m000004.ovf'
r=curl.Curl(f)
r.loadData(f).reshape3D()

In [13]:
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
def coneSlicing(data):
    '''
    all in one
    对每一个x轴上的切片应用距离限制
    最后得到一个大列表   
    
    data: r.data
    '''
    
    assert len(data)==6,'xyz,mxmymz'
    assert len(data[0].shape)==3,'三维数据'
    clipData=[]
    
    #对x循环    
    for i in range(data[0].shape[2]):
        slic=[v[:,:,i] for v in data]
        
        M0=np.sqrt(slic[3]**2+slic[4]**2+slic[5]**2)>0
        #至少得有五个数据把
        if np.sum(M0,axis=None)>5:
            slici=selectRclip(slic)
            clipData.append(slici)
    return pave(clipData)

In [20]:
def pave(data):
    '''
    铺平数据成二维的。
    以那条线为基准铺平数据将会导致很多不同的效果。
    data:[x,y,z,mx,my,mz] ,二维的，不规则的二维数据
    return :[x,y,z,mx,my,mz]，二维的
    '''
    assert len(data[0])==6,'每一个都是六个数据，且每个数据长度一样'
    #新数组的大小    
    len_index=np.max([len(i[0]) for i in data])
    len_col=len(data)
    
    plane=[]

    for xyz in range(6):
        '''
        对变量进行循环
        '''
        planei=np.zeros([len_index,len_col])
        for i,v in enumerate(data):
            '''
            对每一个x轴数据循环
            将数据的中心对准数组的中心
            '''
            youself=v[xyz].shape[0]#第几个切片的xyz变量，的长度
            start=getPaveIndex(youself,lens=170)
            planei[start:start+youself,i]=v[xyz]
        plane.append(planei)
    return plane

In [10]:
def getPaveIndex(youself,lens=170):
    '''
    你的长度比如是50，那么放入170个格的数据中，需要放置的起始点在哪里呢？
    '''
    assert lens>=youself,'数据必须足够大'
    return (lens-youself)//2

In [9]:
def selectRclip(data,rclip=[0,2e-9]):
    '''    
    对平面内的数据进行距离限制，且按照顺时针排序
    
    data：[x,y,z,mx,my,mz]，xyz为二维数据，data应为对X切片的值。
    rclip:[0,2e-9]    
    return :排序后的数据
    '''
    
    assert len(rclip)==2,'长度为2的列表'
    assert len(data[0].shape)==2,'维度为2的数据'
    assert len(set(data[0].flatten()))==1,'x方向应该是单一的'   
    
    datai=[]
    M0=np.sqrt(data[3]**2+data[4]**2+data[5]**2)>0
    for i,v in enumerate(data):
        datai.append(v[M0])
    data=datai
    # 圆环中心
    centery,centerz=data[1].mean(),data[2].mean()
    #距离圆心的距离
    ri=np.sqrt((data[1]-centery)**2+(data[2]-centerz)**2)
    rMax=ri.max()
    #
    selectri=(ri>(rMax-np.max(rclip))) & (ri<(rMax-np.min(rclip)))
    
    for i,v in enumerate(data):
        data[i]=v[selectri]
    #排序
    index=sortByTheta(data[1:3],[centery,centerz])
    
    for i,v in enumerate(data):
        data[i]=v[index]
    return data
    

In [17]:
def sortByTheta(data,center):
    '''
    
    排序，与Y轴正向夹角的顺时针
    data:y,z 返回值
    center:[centerx=y,centerz]  
    return [sorted(y),sorted(z)]
    '''
    
    #减去平均值可以方便的得到以圆柱为中心的
    cy=data[0]-center[0]
    cz=data[1]-center[1]
    #用arccos函数计算每一点与y轴正向的夹角
    thetaY=np.arccos(cy/np.sqrt(cy**2+cz**2))
    thetaY[cz<0]=2*np.pi-thetaY[cz<0]
    #从小到大的排列方式的位置索引。这个索引的作用对象是已经用距离筛选过的数据
    index_thetaY=np.argsort(thetaY)
    #return [data[0][index_thetaY],data[1][index_thetaY]]
    return index_thetaY
